In [1]:
from ud_class import Model
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import re
import pymorphy2
from tqdm import tqdm
import operator
from sklearn.metrics import *

import matplotlib.pyplot as plt

from string import punctuation
full_punctuation = punctuation + "–" + "," + "»" + "«" + "…" +'’'

In [2]:
model = Model('./colloc/russian-syntagrus-ud-2.0-170801.udpipe')


In [3]:
texts_lenta = pd.read_csv("./colloc/3000.csv")

texts_lenta.iloc[0]['texts_3000']

'Указом президента России Бориса Ельцина внесены изменения в  существующую структуру Федеральной службы безопасности РФ, утвержденную в июле прошлого года. Как говорится в поступившем сегодня сообщении Центра общественных связей ФСБ, в соответствии с Основами (Концепцией) государственной политики Российской Федерации по военномустроительству на период до 2005 года, на базе Департамента по борьбе с терроризмом и Управления конституционной безопасности ФСБ создан Департамент по защите конституционного строя и борьбе с терроризмом. В составе департамента организуются три управления с четко определенной компетенцией. В ФСБ отмечают, что "в современных условиях для российскойгосударственности имеют приоритетное значение вопросы защитыконституционного строя, сохранения целостности страны, борьбыс терроризмом и всеми формами экстремизма, а также разведывательно-подрывной деятельностью спецслужб и организаций иностранных государств". Как подчеркивается в сообщении, "органам безопасности в реше

In [4]:
def get_conllu_from_unite_line_text(text, model):
    sentences = model.tokenize(text)
    for s in sentences:
        model.tag(s)
        model.parse(s)
    conllu = model.write(sentences, "conllu")
    return conllu
    
def get_conllu_text_map(conllu_parsed_object):
    conllu_text_map = []
    conllu_sentence_map = []
    for line in conllu_parsed_object.split('\n'):
        if line:
            if line[0].isdigit():
                #print(line.split('\t'))
                conllu_sentence_map.append(line.split('\t'))
            else:
                if(len(conllu_sentence_map) > 0):
                    conllu_text_map.append(conllu_sentence_map)
                    conllu_sentence_map = []   
                    #print("appended")
    if(len(conllu_sentence_map) > 0):
        conllu_text_map.append(conllu_sentence_map)
    return conllu_text_map
    
def get_lemm_and_orig_text_from_udmap(conllu_map):
    lemm_sentences_list = []
    sentences_list = []
    for sentence in conllu_map:
        lemm_line = ''
        line = ''
        for word in sentence: 
            if (word[3] != 'PUNCT'):
                #print(word[2])
                clean_lemma = ''
                for char in word[2]:
                    if char not in full_punctuation:
                        clean_lemma += char.lower()
                lemm_line += clean_lemma + ' '
                line += word[1] + ' '
        
        lemm_sentences_list.append(lemm_line.strip())
        sentences_list.append(line.strip())
        #print()
    return lemm_sentences_list, sentences_list

In [5]:
def ud_lemmatize(text, model):
    conllu = get_conllu_from_unite_line_text(text, model)
    conllu_text_map = get_conllu_text_map(conllu)
    lemm_sent, sentences_list = get_lemm_and_orig_text_from_udmap(conllu_text_map)
    lemm_sent_string = ''
    for l_s in lemm_sent:
        lemm_sent_string += l_s + ' '
    lemm_sent_string = lemm_sent_string.strip()
    #print(lemm_sent_string)
    return lemm_sent_string
ud_lemmatize(texts_lenta.iloc[0]['texts_3000'], model)

'указ президент россия борис ельцин вносить изменение в существовать структура федеральный служба безопасность рф утверждать в июль прошлый год как говорить в поступивать сегодня сообщение центр общественный связь фсб в соответствие с основа концепция государственный политика российский федерация по военномустроительство на период до 2005 год на база департамент по борьба с терроризм и управление конституционный безопасность фсб создавать департамент по защита конституционный строй и борьба с терроризм в состав департамент организовывать три управление с четко определять компетенция в фсб отмечать что в современный условие для российскойгосударственность иметь приоритетный значение вопрос защитыконституционный строй сохранение целостность страна борьбысяча терроризм и весь форма экстремизм а также разведывательно подрывной деятельность спецслужба и организация иностранный государство как подчеркивать в сообщение орган безопасность в решение данные проблема отводить особый роль'

In [3]:
def lemmatize(split_text,morph):
    """
    составляем словарь лемматизированных существительных и словарь всех лемм 
    """
    lemm_split_text = []
    for word in split_text:
        clean_word = ''
        for char in word:
            if char not in full_punctuation:
                clean_word += char.lower()
        parsed_word = morph.parse(clean_word)[0]
        lemma = parsed_word.normal_form
        lemm_split_text.append(lemma)
    lemm_split_text = ' '.join(lemm_split_text)
    return lemm_split_text

In [6]:
processed_lemm_text = []
for ind in tqdm(range(len(texts_lenta))):
    lemm_split_text = ud_lemmatize(texts_lenta.iloc[ind]['texts_3000'], model)
    processed_lemm_text.append(lemm_split_text)

100%|██████████████████████████████████████████| 898/898 [05:33<00:00,  2.69it/s]


In [24]:
processed_lemm_text[0]

'указ президент россия борис ельцин вносить изменение в существовать структура федеральный служба безопасность рф утверждать в июль прошлый год как говорить в поступивать сегодня сообщение центр общественный связь фсб в соответствие с основа концепция государственный политика российский федерация по военномустроительство на период до 2005 год на база департамент по борьба с терроризм и управление конституционный безопасность фсб создавать департамент по защита конституционный строй и борьба с терроризм в состав департамент организовывать три управление с четко определять компетенция в фсб отмечать что в современный условие для российскойгосударственность иметь приоритетный значение вопрос защитыконституционный строй сохранение целостность страна борьбысяча терроризм и весь форма экстремизм а также разведывательно подрывной деятельность спецслужба и организация иностранный государство как подчеркивать в сообщение орган безопасность в решение данные проблема отводить особый роль'

In [9]:
def get_tf_idf_dict(lemm_text_list, save_to_csv = False):
    vect = TfidfVectorizer(ngram_range = (2,4))#stop_words = russian_stopwords
    tfidf_matrix = vect.fit_transform(lemm_text_list)
    df = pd.DataFrame(tfidf_matrix.toarray(), columns = vect.get_feature_names())
    #print(df.head())
    if (save_to_csv): df.to_csv("./text_0_tfidf.csv", sep = '\t')
    tf_idf_dict = df.to_dict()
    return tf_idf_dict

In [7]:
import pandas as pd

In [ ]:
tfidf_dct = pd.read_csv("text_0_tfidf.csv", sep = "\t")
tfidf_dct.head()

In [8]:
dct = get_tf_idf_dict(processed_lemm_text,save_to_csv = True )

MemoryError: 

In [12]:
dct['указ президент россия']

{0: 0.047614240855614935,
 1: 0.0,
 2: 0.0,
 3: 0.0,
 4: 0.0,
 5: 0.0,
 6: 0.0,
 7: 0.0,
 8: 0.0,
 9: 0.0,
 10: 0.0,
 11: 0.0,
 12: 0.0,
 13: 0.0,
 14: 0.0,
 15: 0.0,
 16: 0.0,
 17: 0.0,
 18: 0.0,
 19: 0.0,
 20: 0.0,
 21: 0.0,
 22: 0.0,
 23: 0.0,
 24: 0.0,
 25: 0.0,
 26: 0.04419255588160112,
 27: 0.0,
 28: 0.0,
 29: 0.0,
 30: 0.0,
 31: 0.0,
 32: 0.0,
 33: 0.0,
 34: 0.0,
 35: 0.0,
 36: 0.0,
 37: 0.0,
 38: 0.0,
 39: 0.0,
 40: 0.0,
 41: 0.0,
 42: 0.0,
 43: 0.0,
 44: 0.0,
 45: 0.0,
 46: 0.0,
 47: 0.0,
 48: 0.0,
 49: 0.0,
 50: 0.0,
 51: 0.0,
 52: 0.0,
 53: 0.0,
 54: 0.0,
 55: 0.0,
 56: 0.0,
 57: 0.0,
 58: 0.0,
 59: 0.0,
 60: 0.0,
 61: 0.0,
 62: 0.0,
 63: 0.0,
 64: 0.0,
 65: 0.0,
 66: 0.0,
 67: 0.0,
 68: 0.0,
 69: 0.0,
 70: 0.0,
 71: 0.0,
 72: 0.0,
 73: 0.0,
 74: 0.0,
 75: 0.0,
 76: 0.0,
 77: 0.0,
 78: 0.0,
 79: 0.0,
 80: 0.0,
 81: 0.0,
 82: 0.0,
 83: 0.0,
 84: 0.0,
 85: 0.0,
 86: 0.0,
 87: 0.0,
 88: 0.0,
 89: 0.0,
 90: 0.0,
 91: 0.0,
 92: 0.0,
 93: 0.0,
 94: 0.0,
 95: 0.0,
 96: 0.0,
 97: 0.